In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import json
import time
from datetime import time
import matplotlib.pyplot as plt
import pandas as pd

ModuleNotFoundError: No module named 'requests'

In [ ]:
#Config
BASE_URL = "https://www.prezunic.com.br/"

PARAMS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'DNT': '1',
    'Service-Worker-Navigation-Preload': 'true',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# Criando DataFrame para receber os produtos
df_produtos = pd.DataFrame(columns=['Nome','Valor','URL Image','Categoria'])

# prezunic-mega-menu-0-x-menuDepartmentWrapper

In [ ]:
def getItems(categoria):
    n_items = 0
    for x in range(1, 5):

        # print("Pesquisando Categoria: "+categoria+" Pág.: "+str(x))

        url = BASE_URL+str(categoria)+"?page="+str(x)
        if x == 1:
            url = BASE_URL+str(categoria)

        page = requests.get(url = url, headers = PARAMS)
        soup = BeautifulSoup(page.content, "lxml")

        notFound = soup.find_all("div", {"class": "vtex-search-result-3-x-searchNotFound flex flex-column-s flex-row-ns justify-center-ns items-center h-auto-s h5-ns"})

        if (notFound):
          break

        items = soup.find_all("div", {"class": "vtex-search-result-3-x-galleryItem vtex-search-result-3-x-galleryItem--normal vtex-search-result-3-x-galleryItem--default pa4"})

        for a in items:
            try:
                #Obtem os dados do produto
                nameItem = a.find_all("span", {"class": "prezunic-prezunic-components-0-x-ProductName"})[0].contents[0]
                priceValue = a.find_all("span", {"class": "prezunic-prezunic-components-0-x-currencyContainer"})[1].get_text()
                imageSRC = a.find("img")["src"]
                urlItem = a.find("a")["href"]

                #Insere item no DataFrame
                df_produtos.loc[len(df_produtos)] = [nameItem, priceValue, imageSRC, categoria]
                n_items += 1

            except:
                #print("erro")
                ss = 0
    print("Categoria: "+categoria+" - Items capturados.: "+str(n_items))

In [ ]:
# Get Departamentos do Mercado
df_produtos = pd.DataFrame(columns=['Nome','Valor','URL Image','Categoria'])
# departamentos = ['bazar','bebida-alcoolica','bebida-nao-alcoolica','cama-mesa-e-banho','carnes-e-aves','congelados','frios-e-laticinios','higiene-e-beleza','hortifruti','limpeza','mercearia','padaria']
departamentos = ['mercearia','padaria']

# Iteração por Seções do Mercado
for departamento in departamentos:
  getItems(departamento)

Categoria: mercearia - Items capturados.: 32
Categoria: padaria - Items capturados.: 32


In [ ]:
#Print dos Items Capturados
df_produtos

,Nome,Valor,URL Image,Categoria
0,Leite Uht Itambé Integral Caixa c/ Tampa 1l,"R$ 3,99",https://prezunic.vtexassets.com/arquivos/ids/1...,mercearia
1,Leite Condensado Italac Tetra Pak 395g,"R$ 5,49",https://prezunic.vtexassets.com/arquivos/ids/1...,mercearia
2,Tapioca da Terrinha 500g,"R$ 8,19",https://prezunic.vtexassets.com/arquivos/ids/1...,mercearia
3,Macarrão de Sêmola Parafuso Santa Amália 500g,"R$ 3,99",https://prezunic.vtexassets.com/arquivos/ids/1...,mercearia
4,Feijão Preto Carreteiro Tipo1 1Kg,"R$ 7,95",https://prezunic.vtexassets.com/arquivos/ids/1...,mercearia
...,...,...,...,...
59,Bolo Plusvita Gotas de Chocolate 250g,"R$ 10,69",https://prezunic.vtexassets.com/arquivos/ids/1...,padaria
60,Pao Português J.Caseiro Assado,"R$ 3,29",https://prezunic.vtexassets.com/arquivos/ids/1...,padaria
61,Pão Tipo Tortilha Rap10 Integral Chia/Quinoa 264g,"R$ 13,99",https://prezunic.vtexassets.com/arquivos/ids/1...,padaria
62,Pão de Forma Plusvita Integral s/ Açúcar 350g,"R$ 14,49",https://prezunic.vtexassets.com/arquivos/ids/1...,padaria


In [ ]:
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from urllib.request import urlretrieve
from PIL import Image
from io import BytesIO
import requests

X = []
y = []

for index, row in df_produtos.iterrows():
    response = requests.get(row['URL Image'])
    img = Image.open(BytesIO(response.content))

    # Check if the image has an alpha channel (transparency)
    if img.mode == 'RGBA':
        # Convert to RGB, removing the alpha channel
        img = img.convert('RGB')

    img_resized = img.resize((64, 64))
    img_array = np.array(img_resized).flatten()
    
    X.append(img_array)
    y.append(row['Nome'])

X = np.array(X)
y = np.array(y)

# Treinando o modelo
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X, y)


In [ ]:
#predict from file

local_image_path = 'biscoito.png'
image = cv2.imread(local_image_path)

image_resized = cv2.resize(image, (64, 64))
image_flattened = image_resized.flatten().reshape(1, -1)

predicted_label = model.predict(image_flattened)

print("Previsão:", predicted_label)

font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(image, f'Previsao: {predicted_label[0]}', (10, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
cv2.imshow('Busca Produto', image)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Captura de webcam

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    img_resized = cv2.resize(frame, (64, 64))
    img_array = np.array(img_resized).flatten().reshape(1, -1)

    # Fazendo previsão
    prediction = model.predict(img_array)

    # Mostrar o resultado na tela
    cv2.putText(frame, str(prediction), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Reconhecimento de Produto', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()